In [1]:
import pandas as pd
import numpy as np
from math import pi, cos, sin, sqrt
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
# Load pylab
%pylab inline

Populating the interactive namespace from numpy and matplotlib


C:\Users\jordan\Anaconda2\lib\site-packages\IPython\core\magics\pylab.py:161: UserWarning: pylab import has clobbered these variables: ['cos', 'sqrt', 'pi', 'sin']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
#Our loss evaluation
def log_loss(y_true, y_pred):
    """
    Parameters
    ----------
    y_true : array, shape = [n_samples]
    y_pred : array, shape = [n_samples]

    Returns
    -------
    loss : float
    """
    prediction = sqrt(np.average((log(y_true.astype(int) + 1)  - log(y_pred.astype(int) + 1)) ** 2))
    return prediction

In [3]:
'''# Load the bike data
X = np.loadtxt('data/train.csv',  delimiter=',', 
               skiprows=1, usecols=range(0,13),converters = {1: datestr2num})
y = np.loadtxt('data/train.csv',  delimiter=',', 
               skiprows=1, usecols=[14])
X_real_test = np.loadtxt('data/test.csv',  delimiter=',', 
               skiprows=1, usecols=range(0,13),converters = {1: datestr2num})'''

"# Load the bike data\nX = np.loadtxt('data/train.csv',  delimiter=',', \n               skiprows=1, usecols=range(0,13),converters = {1: datestr2num})\ny = np.loadtxt('data/train.csv',  delimiter=',', \n               skiprows=1, usecols=[14])\nX_real_test = np.loadtxt('data/test.csv',  delimiter=',', \n               skiprows=1, usecols=range(0,13),converters = {1: datestr2num})"

In [4]:
test_df = pd.read_csv("data/test.csv",header=0)
train_df = pd.read_csv("data/train.csv",header=0)
del train_df['dteday'],train_df['holiday'], train_df['atemp'], train_df['workingday']
'''from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
print y_test'''

'from sklearn.cross_validation import train_test_split\nX_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)\nprint y_test'

In [5]:
train_df.corr()

,instant,season,yr,mnth,hr,weekday,weathersit,temp,hum,windspeed,cnt
instant,1.000000,0.479463,0.866974,0.493368,-0.005638,-0.016307,-0.005118,0.180773,0.032624,-0.086933,0.310185
season,0.479463,1.000000,-0.004797,0.971524,-0.006546,-0.009691,0.008879,0.258689,0.190610,-0.147121,0.163439
yr,0.866974,-0.004797,1.000000,-0.004932,-0.004234,-0.018162,-0.012548,0.061226,-0.078606,-0.015221,0.260403
mnth,0.493368,0.971524,-0.004932,1.000000,-0.006818,-0.001012,0.012144,0.257589,0.204537,-0.150192,0.166862
hr,-0.005638,-0.006546,-0.004234,-0.006818,1.000000,-0.001660,-0.022740,0.145430,-0.278011,0.146631,0.400601
weekday,-0.016307,-0.009691,-0.018162,-0.001012,-0.001660,1.000000,-0.016439,-0.026143,-0.059722,-0.006178,0.027690
weathersit,-0.005118,0.008879,-0.012548,0.012144,-0.022740,-0.016439,1.000000,-0.055035,0.406244,0.007261,-0.128655
temp,0.180773,0.258689,0.061226,0.257589,0.145430,-0.026143,-0.055035,1.000000,-0.064949,-0.017852,0.394454
hum,0.032624,0.190610,-0.078606,0.204537,-0.278011,-0.059722,0.406244,-0.064949,1.000000,-0.318607,-0.317371
windspeed,-0.086933,-0.147121,-0.015221,-0.150192,0.146631,-0.006178,0.007261,-0.017852,-0.318607,1.000000,0.101369


In [6]:
def binarization(data,column) :
    '''Creation of dummy variables'''
    for unique in data[column].unique() :
        data[column+'='+str(unique)] = data[column].apply(lambda x : 1 if x == unique else 0)

In [7]:
#Binarization of weekday, season, weathersit
for column in ['season','weathersit'] :
    binarization(train_df,column)
    binarization(test_df,column)

In [8]:
def categorization(data,column) :
    '''Creation of category variables'''
    data[column+'_cat'] = data[column].astype('category')

In [9]:
train_df['year_season'] = (train_df['yr'] + 1 ) * train_df['season']
test_df['year_season'] = (test_df['yr'] + 1 ) * test_df['season']

In [10]:
for column in [el for el in train_df.columns if '=' in el] + ['yr'] + ['year_season']:
    categorization(train_df,column)
    categorization(test_df,column)

In [11]:
train_df['log_cnt']=np.log(1+train_df['cnt'])


In [12]:

train_df.dtypes

instant                int64
season                 int64
yr                     int64
mnth                   int64
hr                     int64
weekday                int64
weathersit             int64
temp                 float64
hum                  float64
windspeed            float64
cnt                    int64
season=1               int64
season=2               int64
season=3               int64
season=4               int64
weathersit=1           int64
weathersit=2           int64
weathersit=3           int64
weathersit=4           int64
year_season            int64
season=1_cat        category
season=2_cat        category
season=3_cat        category
season=4_cat        category
weathersit=1_cat    category
weathersit=2_cat    category
weathersit=3_cat    category
weathersit=4_cat    category
yr_cat              category
year_season_cat     category
log_cnt              float64
dtype: object

In [13]:
train_df.tail(10)


,instant,season,yr,mnth,hr,weekday,weathersit,temp,hum,windspeed,...,season=2_cat,season=3_cat,season=4_cat,weathersit=1_cat,weathersit=2_cat,weathersit=3_cat,weathersit=4_cat,yr_cat,year_season_cat,log_cnt
10876,17084,4,1,12,14,3,1,0.42,0.50,0.1940,...,0,0,1,1,0,0,0,1,8,5.389072
10877,17085,4,1,12,15,3,1,0.42,0.50,0.2836,...,0,0,1,1,0,0,0,1,8,5.472271
10878,17086,4,1,12,16,3,1,0.42,0.50,0.3582,...,0,0,1,1,0,0,0,1,8,5.814131
10879,17087,4,1,12,17,3,1,0.40,0.50,0.3881,...,0,0,1,1,0,0,0,1,8,6.333280
10880,17088,4,1,12,18,3,1,0.38,0.50,0.3582,...,0,0,1,1,0,0,0,1,8,6.345636
10881,17089,4,1,12,19,3,1,0.38,0.50,0.3881,...,0,0,1,1,0,0,0,1,8,5.820083
10882,17090,4,1,12,20,3,1,0.36,0.57,0.2239,...,0,0,1,1,0,0,0,1,8,5.488938
10883,17091,4,1,12,21,3,1,0.34,0.61,0.2239,...,0,0,1,1,0,0,0,1,8,5.129899
10884,17092,4,1,12,22,3,1,0.34,0.61,0.0896,...,0,0,1,1,0,0,0,1,8,4.867534
10885,17093,4,1,12,23,3,1,0.32,0.66,0.1343,...,0,0,1,1,0,0,0,1,8,4.488636


In [14]:
#Standardization of numerical features
def standardization(data,column) :
    '''Standardization of our numerical features'''
    mean_col = data[column].mean()
    std = data[column].std()
    data[column+'_std'] = data[column].apply(lambda x: (x - mean_col)/std)

In [15]:
for column in ['windspeed','hum','temp'] :
    standardization(train_df,column)
    standardization(test_df,column)

In [16]:
#Cyclic featuring: Useful for dates: for example we want 'Monday' to be close to 'Sunday' => cosine
def cyclic_featuring(data) :
    '''Cyclic features'''
    d_day = {'0' : 0, '1': 2*pi/7, '2' : 4*pi/7, '3' : 6*pi/7, '4' : 8*pi/7, '5' : 10*pi/7,
     '6' : 12*pi/7}
    
    d_hour = {}
    for h in range(25) :
        d_hour[h] = 2*pi*h/24
    
    d_month = {}
    for m in range(1,13) :
        d_month[m] = 2*pi*m/12

    data['cosDay'] = data['weekday'].apply(lambda x: cos(d_day[str(x)]))
    data['sinDay'] = data['weekday'].apply(lambda x: sin(d_day[str(x)]))
    data['cosHour'] = data['hr'].apply(lambda x: cos(d_hour[x]))
    data['sinHour'] = data['hr'].apply(lambda x: sin(d_hour[x]))
    data['cosMonth'] = data['mnth'].apply(lambda x: cos(d_month[x]))
    data['sinMonth'] = data['mnth'].apply(lambda x: sin(d_month[x]))

In [17]:
cyclic_featuring(train_df)
cyclic_featuring(test_df)

In [18]:
train_df.head(10)

,instant,season,yr,mnth,hr,weekday,weathersit,temp,hum,windspeed,...,log_cnt,windspeed_std,hum_std,temp_std,cosDay,sinDay,cosHour,sinHour,cosMonth,sinMonth
0,1,1,0,1,0,6,1,0.24,0.81,0.0000,...,2.833213,-1.567682,0.993167,-1.333599,0.62349,-0.781831,1.000000e+00,0.000000,0.866025,0.5
1,2,1,0,1,1,6,1,0.22,0.80,0.0000,...,3.713572,-1.567682,0.941206,-1.438841,0.62349,-0.781831,9.659258e-01,0.258819,0.866025,0.5
2,3,1,0,1,2,6,1,0.22,0.80,0.0000,...,3.496508,-1.567682,0.941206,-1.438841,0.62349,-0.781831,8.660254e-01,0.500000,0.866025,0.5
3,4,1,0,1,3,6,1,0.24,0.75,0.0000,...,2.639057,-1.567682,0.681399,-1.333599,0.62349,-0.781831,7.071068e-01,0.707107,0.866025,0.5
4,5,1,0,1,4,6,1,0.24,0.75,0.0000,...,0.693147,-1.567682,0.681399,-1.333599,0.62349,-0.781831,5.000000e-01,0.866025,0.866025,0.5
5,6,1,0,1,5,6,2,0.24,0.75,0.0896,...,0.693147,-0.832404,0.681399,-1.333599,0.62349,-0.781831,2.588190e-01,0.965926,0.866025,0.5
6,7,1,0,1,6,6,1,0.22,0.80,0.0000,...,1.098612,-1.567682,0.941206,-1.438841,0.62349,-0.781831,6.123234e-17,1.000000,0.866025,0.5
7,8,1,0,1,7,6,1,0.20,0.86,0.0000,...,1.386294,-1.567682,1.252975,-1.544083,0.62349,-0.781831,-2.588190e-01,0.965926,0.866025,0.5
8,9,1,0,1,8,6,1,0.24,0.75,0.0000,...,2.197225,-1.567682,0.681399,-1.333599,0.62349,-0.781831,-5.000000e-01,0.866025,0.866025,0.5
9,10,1,0,1,9,6,1,0.32,0.76,0.0000,...,2.708050,-1.567682,0.733360,-0.912633,0.62349,-0.781831,-7.071068e-01,0.707107,0.866025,0.5


In [19]:
features =[el for el in train_df.columns if 'cat' in el] + [col for col in train_df.columns if 'std' in col] + [col for col in train_df.columns if 'sin' in col] + [col for col in train_df if 'cos' in col]

#We can select the feature either by statistical filtering....
select = SelectKBest(f_classif, k='all')
X_new = pd.DataFrame(select.fit_transform(train_df.loc[:,features], train_df.cnt))

features_selected=[]
for element in features :
    if select.get_support()[features.index(element)] :
        features_selected.append(element)

In [20]:
print features_selected
train_df.describe()

['season=1_cat', 'season=2_cat', 'season=3_cat', 'season=4_cat', 'weathersit=1_cat', 'weathersit=2_cat', 'weathersit=3_cat', 'weathersit=4_cat', 'yr_cat', 'year_season_cat', 'windspeed_std', 'hum_std', 'temp_std', 'sinDay', 'sinHour', 'sinMonth', 'cosDay', 'cosHour', 'cosMonth']


,instant,season,yr,mnth,hr,weekday,weathersit,temp,hum,windspeed,...,log_cnt,windspeed_std,hum_std,temp_std,cosDay,sinDay,cosHour,sinHour,cosMonth,sinMonth
count,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000,...,10886.000000,1.088600e+04,1.088600e+04,1.088600e+04,10886.000000,10886.000000,1.088600e+04,10886.000000,1.088600e+04,1.088600e+04
mean,8537.421183,2.506614,0.501929,6.521495,11.541613,2.998622,1.418427,0.493436,0.618865,0.191036,...,4.591364,1.028625e-13,4.089840e-14,-4.802770e-15,0.004992,-0.001554,-2.834917e-03,-0.003772,-2.720442e-03,-3.009216e-03
std,5026.363303,1.116174,0.500019,3.444373,6.915838,2.007770,0.633839,0.190039,0.192450,0.121859,...,1.419454,1.000000e+00,1.000000e+00,1.000000e+00,0.708826,0.705429,7.072873e-01,0.706975,7.073613e-01,7.069055e-01
min,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.020000,0.000000,0.000000,...,0.693147,-1.567682e+00,-3.215711e+00,-2.491258e+00,-0.900969,-0.974928,-1.000000e+00,-1.000000,-1.000000e+00,-1.000000e+00
25%,4282.250000,2.000000,0.000000,4.000000,6.000000,1.000000,1.000000,0.340000,0.470000,0.104500,...,3.761200,-7.101315e-01,-7.735222e-01,-8.073910e-01,-0.900969,-0.781831,-7.071068e-01,-0.707107,-8.660254e-01,-8.660254e-01
50%,8666.500000,3.000000,1.000000,7.000000,12.000000,3.000000,1.000000,0.500000,0.620000,0.194000,...,4.983607,2.432527e-02,5.899721e-03,3.454240e-02,-0.222521,0.000000,-1.836970e-16,0.000000,-1.836970e-16,-2.449294e-16
75%,13016.750000,4.000000,1.000000,10.000000,18.000000,5.000000,2.000000,0.640000,0.770000,0.253700,...,5.652489,5.142367e-01,7.853216e-01,7.712342e-01,0.623490,0.781831,7.071068e-01,0.707107,5.000000e-01,5.000000e-01
max,17093.000000,4.000000,1.000000,12.000000,23.000000,6.000000,4.000000,1.000000,1.000000,0.850700,...,6.885510,5.413351e+00,1.980435e+00,2.665584e+00,1.000000,0.974928,1.000000e+00,1.000000,1.000000e+00,1.000000e+00


In [26]:
'''The methods'''

LNR = LogisticRegression(C = 1e4, n_jobs=-1)
LR = LogisticRegression(C = 50, n_jobs=-1)
kNN = KNeighborsClassifier(n_neighbors=5500, n_jobs=-1)
RF = RandomForestClassifier(n_jobs=-1, n_estimators=250)
svm = SVC(C= 0.01, probability=True)

In [22]:
def cross_validate(design_matrix, cnt, method, cv_folds):
    pred = np.zeros(cnt.shape)
    matrix_log_loss = np.zeros(1)
    for tr, te in cv_folds:
        # Restrict data to train/test folds
        Xtr = design_matrix[tr, :]
        ytr = labels[tr]
        Xte = design_matrix[te, :]
        #print Xtr.shape, ytr.shape, Xte.shape
        # Fit classifier
        method.fit(Xtr, ytr)

        # Predict probabilities (of belonging to +1 class) on test data
        yte_pred = method.predict(Xte) # two-dimensional array
        # Identify the index, in yte_pred, of the positive class (y=1)
        print log_loss(ytr,yte_pred)
        matrix_log_loss[te] = log_loss(ytr,yte_pred)
        pred[te] = yte_pred
    return pred,matrix_log_loss

In [24]:
from sklearn.metrics import make_scorer
loss  = make_scorer(log_loss, greater_is_better=False)
score = make_scorer(log_loss, greater_is_better=True)

In [28]:
from sklearn.model_selection import cross_val_score
from sklearn import metrics
scores = cross_val_score(
     RF, train_df.loc[:,features_selected], train_df.cnt, cv=5, scoring=score,verbose=1)
scores

JoblibMemoryError: JoblibMemoryError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
C:\Users\jordan\Anaconda2\lib\runpy.py in _run_module_as_main(mod_name='ipykernel.__main__', alter_argv=1)
    169     pkg_name = mod_name.rpartition('.')[0]
    170     main_globals = sys.modules["__main__"].__dict__
    171     if alter_argv:
    172         sys.argv[0] = fname
    173     return _run_code(code, main_globals, None,
--> 174                      "__main__", fname, loader, pkg_name)
        fname = r'C:\Users\jordan\Anaconda2\lib\site-packages\ipykernel\__main__.py'
        loader = <pkgutil.ImpLoader instance>
        pkg_name = 'ipykernel'
    175 
    176 def run_module(mod_name, init_globals=None,
    177                run_name=None, alter_sys=False):
    178     """Execute a module's code without importing it

..........................................

In [60]:
'''Fit method'''
#Chose your estimator (LR, LNR, kNN, RF,svm)
estimator = LR
cross_validate (train_df.loc[:,features_selected],train_df.cnt,estimator,10)

TypeError: 'int' object is not iterable

In [ ]:
estimator.score(train_df.loc[:,features_selected], train_df.cnt)

In [ ]:
log_loss(train_df.cnt, est)